In [ ]:

# Efficient Frontier GUI (No Sector Constraints / No Plotting)

import pandas as pd
import numpy as np
from datetime import datetime, date
from io import BytesIO
from ipywidgets import FileUpload, Button, VBox, Output, Label, SelectMultiple, HBox, Text, FloatSlider
from IPython.display import display
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
import copy

upload = FileUpload(accept='.xlsx', multiple=False)
indices_select = SelectMultiple(description='Assets:', rows=10)
expected_returns_box = VBox()

decay_slider = FloatSlider(value=0.99, min=0.90, max=0.999, step=0.01, description='EWMA Decay:')
run_button = Button(description="Run Optimizer", button_style='success')
output = Output()

expected_return_inputs = {}

def log_returns(df):
    return np.log(df / df.shift(1)).dropna()

def ewma_covariance_matrix(X, alpha):
    T, D = X.shape
    cov = np.zeros((D, D))
    for tt in range(T):
        x_t = X[tt, :]
        if tt == 0:
            cov = x_t[:, np.newaxis] @ x_t[np.newaxis, :]
        else:
            cov = alpha * cov + (1 - alpha) * x_t[:, np.newaxis] @ x_t[np.newaxis, :]
    return cov

def get_single_target_weights(target_return, ef):
    ef_i = copy.deepcopy(ef)
    ef_i.efficient_return(target_return)
    return ef_i.weights

def get_single_target_metrics(target_return, ef):
    ef_i = copy.deepcopy(ef)
    ef_i.efficient_return(target_return)
    return ef_i.portfolio_performance()

def update_asset_fields(change):
    global expected_return_inputs
    selected = list(indices_select.value)
    expected_return_inputs = {}
    er_rows = []

    for name in selected:
        label = Label(value=name, layout={'width': '200px'})
        er_input = Text(placeholder='Expected Return', layout={'width': '100px'})
        expected_return_inputs[name] = er_input
        er_rows.append(HBox([label, er_input]))

    expected_returns_box.children = er_rows

def run_optimizer(change):
    output.clear_output()
    with output:
        if not upload.value or not indices_select.value:
            print("Please upload data and select assets.")
            return

        try:
            file_info = upload.value[0]
            content = BytesIO(file_info['content'])
            df = pd.read_excel(content, sheet_name="PyData", engine='openpyxl')
            df = df.rename(columns=df.iloc[0]).drop(0)
            df['Dates'] = pd.to_datetime(df['Dates'])
            df = df.set_index("Dates")
            df = df.astype(float)

            assets = list(indices_select.value)
            df = df[assets]
            df = df[df.index > datetime(2017, 1, 1)]
            logret = log_returns(df)
            decay = decay_slider.value
            cov = ewma_covariance_matrix(logret.values, alpha=decay)
            cov = pd.DataFrame(cov * 50, index=logret.columns, columns=logret.columns)

            mu = {}
            for name in assets:
                mu[name] = float(expected_return_inputs[name].value.strip() or 0)
            mu = pd.Series(mu)

            ef = EfficientFrontier(mu, cov, weight_bounds=(0, 1))
            ef.add_objective(objective_functions.L2_reg, gamma=0.01)

            min_ret, max_ret = ef._max_quadratic_utility_bounds()
            return_range = np.linspace(min_ret, max_ret, 25)

            frontier_weights = [get_single_target_weights(r, ef) for r in return_range]
            frontier_metrics = [get_single_target_metrics(r, ef) for r in return_range]

            weights_df = pd.DataFrame(frontier_weights, columns=mu.index)
            weights_df['Target Return'] = return_range
            metrics_df = pd.DataFrame(frontier_metrics, columns=['Return', 'Volatility', 'Sharpe'])

            today = date.today().strftime("%d%m%y")
            weights_df.to_excel(f"efficient_frontier_weights_{today}.xlsx")
            metrics_df.to_excel(f"efficient_frontier_metrics_{today}.xlsx")

            display(weights_df.head())
            display(metrics_df.head())
            print(f"Files saved: efficient_frontier_weights_{today}.xlsx, efficient_frontier_metrics_{today}.xlsx")

        except Exception as e:
            print("An error occurred:", e)

indices_select.observe(update_asset_fields, names='value')
run_button.on_click(run_optimizer)

VBox([
    Label("📥 Upload Excel file (sheet: PyData):"),
    upload,
    Label("📊 Select assets:"),
    indices_select,
    Label("📈 Expected returns:"),
    expected_returns_box,
    decay_slider,
    run_button,
    output
])
